In [1]:
import subprocess
from pathlib import Path

import fsspec
from utils import sync_notebook

In [2]:
def run_memray(file: str):
    output_file = "results/memray-" + Path(file).stem + ".bin"
    summary_file = "results/memray-" + Path(file).stem + ".json"
    command = ["memray", "run", "--force", "--output", output_file, file]
    subprocess.run(command)
    summary_command = [
        "memray",
        "stats",
        "--force",
        "--json",
        "--output",
        summary_file,
        output_file,
    ]
    subprocess.run(summary_command)

In [3]:
fs = fsspec.filesystem("file")

In [4]:
files = fs.glob("resample-*.ipynb")
for file in files:
    sync_notebook(file)

[jupytext] Reading /home/jovyan/Code/developmentseed/warp-resample-profiling/examples/resample-gdal-netcdf-local.ipynb in format ipynb
[jupytext] Loading /home/jovyan/Code/developmentseed/warp-resample-profiling/examples/resample-gdal-netcdf-local.py
[jupytext] Unchanged /home/jovyan/Code/developmentseed/warp-resample-profiling/examples/resample-gdal-netcdf-local.ipynb
[jupytext] Unchanged /home/jovyan/Code/developmentseed/warp-resample-profiling/examples/resample-gdal-netcdf-local.py
[jupytext] Reading /home/jovyan/Code/developmentseed/warp-resample-profiling/examples/resample-gdal-netcdf-vsis3.ipynb in format ipynb
[jupytext] Loading /home/jovyan/Code/developmentseed/warp-resample-profiling/examples/resample-gdal-netcdf-vsis3.py
[jupytext] Unchanged /home/jovyan/Code/developmentseed/warp-resample-profiling/examples/resample-gdal-netcdf-vsis3.ipynb
[jupytext] Unchanged /home/jovyan/Code/developmentseed/warp-resample-profiling/examples/resample-gdal-netcdf-vsis3.py
[jupytext] Reading /

In [5]:
files = fs.glob("resample-*.py")
for file in files:
    run_memray(file)

Writing profile results into results/memray-resample-gdal-netcdf-local.bin
[memray] Successfully generated profile results.

You can now generate reports from the stored allocation records.
Some example commands to generate reports:

/opt/conda/bin/python -m memray flamegraph results/memray-resample-gdal-netcdf-local.bin
  Computing statistics... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:005m 28% -:--:--
Wrote results/memray-resample-gdal-netcdf-local.json
Writing profile results into results/memray-resample-gdal-netcdf-vsis3.bin
[memray] Successfully generated profile results.

You can now generate reports from the stored allocation records.
Some example commands to generate reports:

/opt/conda/bin/python -m memray flamegraph results/memray-resample-gdal-netcdf-vsis3.bin
  Computing statistics... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:005m 93% 0:00:01
Wrote results/memray-resample-gdal-netcdf-vsis3.json
Writing profile results into results/memray-resample-gdal-vr

The input area chunks are large. This usually means that the input area is of much higher resolution than the output area. You can reduce the chunks passed, and ponder whether you are using the right resampler for the job.
The input area chunks are large. This usually means that the input area is of much higher resolution than the output area. You can reduce the chunks passed, and ponder whether you are using the right resampler for the job.


Writing profile results into results/memray-resample-pyresample-h5netcdf-.bin
[memray] Successfully generated profile results.

You can now generate reports from the stored allocation records.
Some example commands to generate reports:

/opt/conda/bin/python -m memray flamegraph results/memray-resample-pyresample-h5netcdf-.bin
  Computing statistics... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:005m 95% 0:00:01
Wrote results/memray-resample-pyresample-h5netcdf-.json
Writing profile results into results/memray-resample-rasterio-netcdf-vsis3.bin
[memray] Successfully generated profile results.

You can now generate reports from the stored allocation records.
Some example commands to generate reports:

/opt/conda/bin/python -m memray flamegraph results/memray-resample-rasterio-netcdf-vsis3.bin
  Computing statistics... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00m 0:00:0100:01
Wrote results/memray-resample-rasterio-netcdf-vsis3.json
Writing profile results into results/me